In [1]:
import numpy as np
import cv2
import os
from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from keras.optimizers import Adam
from keras import Model
import time as t

Using TensorFlow backend.


In [2]:
labelsToIndex = {'bird':0, 'cannon':1, 'chandelier':2, 'horse':3, 
        'hot_dog':4, 'panda':5, 'windmill':6}

In [3]:
def onehot(array, max_categories):
    output = []
    for value in array:
        temp = np.zeros(shape=(max_categories,))
        temp[int(value)]=1
        output.append(temp)
    output = np.array(output)
    return output

In [4]:
a = [1, 0, 2 , 3, 2]
a = onehot(a, 4)
print(a)

[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [ ]:
full_array = []
for file in os.listdir('./data/'):
    current_array = np.load('./data/'+ file)
    file_name = file[:-4]
    file_index = labelsToIndex[file_name]
    index_array = np.zeros(shape=(current_array.shape[0],1)) + file_index
    current_array = np.concatenate((current_array, index_array), axis=1)
    print(current_array.shape)
    for i in range(current_array.shape[0]):
        full_array.append(current_array[i])
full_array = np.array(full_array)

In [5]:
full_array = np.load('./data/shuffled_full_array.npy')

In [6]:
print(full_array[0])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.  81. 100.  53.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.  32.  20.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.  57. 253. 255. 255. 209. 106.  14.   0.   0.   0.
   0.   0.   0.   0.   0.   9. 230. 220.  26.   0.   0.   0.   0.   0.
   0.   0.   0.   4. 208. 221.  31.  93. 197. 255. 242.  60.   0.   0.
   0.   0.   0.   0.   0. 112. 255. 241. 213.  21.   0.   0.   0.   0.
   0.   0.   0. 113. 255.  73.   0.   0.   0. 115. 255. 107.   0.   0.
   0.   0.   0.   0.   8. 229. 180.  55. 242. 208.  18.   0.   0.   0.
   0.   0.  28. 242. 172.   0.   0.   4. 119. 249. 200.  14.   0.   0.
   0.   0.   0.   0. 112. 255.  54.   0.  64. 246. 197.   9.   0.   0.
   0.   0. 172. 242.  28.   0.  42. 201. 255. 161.  12.   0.   0.   0.
   0. 

In [7]:
num_testing_examples = 75000

full_training = full_array[num_testing_examples+1:]
full_testing  = full_array[:num_testing_examples]
print(full_training.shape)
print(full_testing.shape)

X_training = full_training[:,:-1]
Y_training = full_training[:, 784]
print(X_training.shape)
print(Y_training.shape)


X_testing = full_testing[:, :-1]
Y_testing = full_testing[:, 784]

print(Y_training)
print(Y_testing)





(962009, 785)
(75000, 785)
(962009, 784)
(962009,)
[6. 5. 6. ... 4. 2. 5.]
[6. 2. 3. ... 6. 3. 1.]


In [8]:
Y_training = onehot(Y_training, 7)
Y_testing  = onehot(Y_testing, 7)
print(Y_training.shape)
print(Y_testing.shape)

In [10]:
X_training /= 255
X_testing  /= 255

In [18]:
nn_input = Input(shape=(784,))

activation = Dense(500)(nn_input)
activation = BatchNormalization()(activation)
activation = Activation('relu')(activation)

middle_layers = [256, 256, 128, 64, 32, 16]
for layer in middle_layers:
    activation = Dense(layer)(activation)
    activation = BatchNormalization()(activation)
    activation = Activation('relu')(activation)

activation = Dense(7)(activation)
output = Activation('softmax')(activation)
    

In [19]:
model = Model(inputs=nn_input, outputs=output)
model.compile(optimizer=Adam(lr=0.001, decay=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
costs = dict()
for i in range(10):
    model.fit(x=X_training, y=Y_training, batch_size=128, epochs=1, verbose=1)
    costs[i] = model.evaluate(x=X_testing, y=Y_testing)

Epoch 1/1
75000/75000 [==============================] - 4s 57us/step
Epoch 1/1
166272/962009 [====>.........................] - ETA: 1:00 - loss: 0.1932 - acc: 0.9377

In [21]:
print(costs)

{0: [0.33638936329126357, 0.8924666666603088], 1: [0.2960185162909826, 0.9048000000063579], 2: [0.2819953625833988, 0.9091066666730245], 3: [0.2725732209114234, 0.9129599999936422], 4: [0.263278889601628, 0.9172133333333333]}
